### KOGPT2를 활용한 학습 실행


In [1]:
#라이브러리 불러오기, pip로 설치하였음
import gluonnlp as nlp #kogpt2에서 사용했어서 사용
from gluonnlp.data import SentencepieceTokenizer #상동
from transformers import TFGPT2LMHeadModel
import tensorflow as tf
import numpy as np

In [2]:
#모델 생성 방식 따라 디렉토리 경로 입력하는 클래스
class GPT2Model(tf.keras.Model):
    def __init__(self, dir_path):
        super(GPT2Model, self).__init__()
        self.gpt2 = TFGPT2LMHeadModel.from_pretrained(dir_path)
        
    def call(self, inputs):
        return self.gpt2(inputs)[0]

In [10]:
#링크에서 사전학습된 모델을 다운받는다
'''
https://www.dropbox.com/s/nzfa9xpzm4edp6o/gpt_ckpt.zip
'''

'\nhttps://www.dropbox.com/s/nzfa9xpzm4edp6o/gpt_ckpt.zip\n'

In [3]:
#모델 위치 설정 및 불러오기
BASE_MODEL_PATH = './gpt_ckpt'
gpt_model = GPT2Model(BASE_MODEL_PATH)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at ./gpt_ckpt.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [4]:
#사전학습 모델 구성하기
BATCH_SIZE = 16
NUM_EPOCHS = 5
MAX_LEN = 40
TOKENIZER_PATH = './gpt_ckpt/gpt2_kor_tokenizer.spiece'

tokenizer = SentencepieceTokenizer(TOKENIZER_PATH, num_best=0, alpha=0)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(TOKENIZER_PATH,
                                               mask_token=None,
                                               sep_token=None,
                                               cls_token=None,
                                               unknown_token='<unk>',
                                               padding_token='<pad>',
                                               bos_token='<s>',
                                               eos_token='</s>')

In [6]:
#토크나이저 & 사전 학습 모델 통해 문장 만들기 함수
def tf_top_k_top_p_filtering(logits, top_k=0, top_p=0.0, filter_value=-99999):
    _logits = logits.numpy()
    top_k = min(top_k, logits.shape[-1])  
    if top_k > 0:
        indices_to_remove = logits < tf.math.top_k(logits, top_k)[0][..., -1, None]
        _logits[indices_to_remove] = filter_value

    if top_p > 0.0:
        sorted_logits = tf.sort(logits, direction='DESCENDING')
        sorted_indices = tf.argsort(logits, direction='DESCENDING')
        cumulative_probs = tf.math.cumsum(tf.nn.softmax(sorted_logits, axis=-1), axis=-1)

        sorted_indices_to_remove = cumulative_probs > top_p
        sorted_indices_to_remove = tf.concat([[False], sorted_indices_to_remove[..., :-1]], axis=0)
        indices_to_remove = sorted_indices[sorted_indices_to_remove].numpy().tolist()
        
        _logits[indices_to_remove] = filter_value
    return tf.constant([_logits])

#seed_word:시작하는 단어
def generate_sent(seed_word, model, max_step=100, greedy=False, top_k=0, top_p=0.):
    sent = seed_word
    toked = tokenizer(sent)
    
    for _ in range(max_step):
        #인풋 = 토큰화된 단어의 인덱스
        input_ids = tf.constant([vocab[vocab.bos_token],]  + vocab[toked])[None, :] 
        #아웃풋 = 문장의 마지막 단어
        outputs = model(input_ids)[:, -1, :]
        if greedy: #가장 확률 높은 단어만 선택해서 반복되는 경우가 잦음
            gen = vocab.to_tokens(tf.argmax(outputs, axis=-1).numpy().tolist()[0])
        else: #그렇지 않음
            output_logit = tf_top_k_top_p_filtering(outputs[0], top_k=top_k, top_p=top_p)
            gen = vocab.to_tokens(tf.random.categorical(output_logit, 1).numpy().tolist()[0])[0]
        if gen == '</s>': #멈춤단어나오면 멈춤
            break
        sent += gen.replace('▁', ' ')
        toked = tokenizer(sent)

    return sent

In [7]:
#사전 학습된 모델 확인
print(generate_sent('지금', gpt_model))
print(generate_sent('고양인', gpt_model))
print(generate_sent('기분이 좋아', gpt_model))

지금 델따 줄<unk>~◈ 스티브W.
고양인우리에서 일할 광교신도시 입주민들로 구성된 ‘인더스밸리3단지’(현대산업개발, 시공사, 삼성물산)다.
기분이 좋아졌어요”라며 환하게 미소 지었다.


In [8]:
#성경 데이터 넣기
sents = [s[:-1] for s in open('edited_bible.txt', encoding='cp949').readlines()]

In [9]:
#토크나이저에 인풋 아웃풋 나눠 넣기
input_data = []
output_data = []

for s in sents:
    tokens = [vocab[vocab.bos_token],]  + vocab[tokenizer(s)] + [vocab[vocab.eos_token],]
    input_data.append(tokens[:-1])
    output_data.append(tokens[1:])

In [10]:
#패딩 해서 입출력 구성
input_data = tf.keras.preprocessing.sequence.pad_sequences(input_data, MAX_LEN, value=vocab[vocab.padding_token])
output_data = tf.keras.preprocessing.sequence.pad_sequences(output_data, MAX_LEN, value=vocab[vocab.padding_token])

input_data = np.array(input_data, dtype=np.int64)
output_data = np.array(output_data, dtype=np.int64)

In [12]:
#손실함수 & 정확도 측정 설정
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, vocab[vocab.padding_token]))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

def accuracy_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, vocab[vocab.padding_token]))
    mask = tf.expand_dims(tf.cast(mask, dtype=pred.dtype), axis=-1)
    pred *= mask    
    acc = train_accuracy(real, pred)

    return tf.reduce_mean(acc)

In [13]:
#모델 컴파일
gpt_model.compile(loss=loss_function,
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=[accuracy_function])

In [14]:
#학습 실행
history = gpt_model.fit(input_data, output_data, 
                    batch_size=BATCH_SIZE, epochs=NUM_EPOCHS)

Epoch 1/5
1945/1945 [==============================] - 4149s 2s/step - loss: 2.5410 - accuracy_function: 0.1838
Epoch 2/5
1945/1945 [==============================] - 4369s 2s/step - loss: 2.2639 - accuracy_function: 0.2067
Epoch 3/5
1945/1945 [==============================] - 3912s 2s/step - loss: 2.1013 - accuracy_function: 0.2197
Epoch 4/5
1945/1945 [==============================] - 3907s 2s/step - loss: 1.9717 - accuracy_function: 0.2304
Epoch 5/5
1945/1945 [==============================] - 3907s 2s/step - loss: 1.8556 - accuracy_function: 0.2400


In [15]:
#저장
import os
DATA_OUT_PATH = './data_out'
model_name = 'bible_model'
save_path = os.path.join(DATA_OUT_PATH, model_name)

if not os.path.exists(save_path):
    os.makedirs(save_path)

gpt_model.gpt2.save_pretrained(save_path)

loaded_gpt_model = GPT2Model(save_path)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at ./data_out\bible_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [18]:
print(generate_sent('프로젝트', gpt_model))
print('-'*32)
print(generate_sent('마라탕 존맛탱', gpt_model))
print('-'*32)
print(generate_sent('어젯밤은 유난히 쌀쌀하여 버티기 어려웠다.', gpt_model))
print('-'*32)

프로젝트 시험 참가자들은 다 여로함 같아 오늘 내가 너희에게 한 것을 생각하되 너희에게 말하노니 너희 가운데서 여인이 머리털로 자랐으므로 여인이 되었도다 하고
--------------------------------
마라탕 존맛탱은 끓는 가루를 밀가루 반죽에 사르는 중이요 온천 깊은 곳까지 드리는 중이요 거룩한 곳까지 드리는 중이요 여호와를 위한
--------------------------------
어젯밤은 유난히 쌀쌀하여 버티기 어려웠다.
--------------------------------
